In [1]:
import gym
import tianshou as ts

import torch

In [2]:
env = gym.make('Acrobot-v1')

In [19]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make('Acrobot-v1') for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make('Acrobot-v1') for _ in range(100)])

In [20]:
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = ts.utils.net.common.Net(state_shape, action_shape, activation='ReLu')
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [21]:
policy = ts.policy.DQNPolicy(net, optim, discount_factor = 0.9, estimation_step = 3, target_update_freq = 320)

In [22]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise = True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise = True)

In [23]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10001it [00:06, 1455.94it/s, env_step=10000, len=500, loss=3.708, n/ep=5, n/st=10, rew=-500.00]                           


Epoch #1: test_reward: -500.000000 ± 0.000000, best_reward: -382.050000 ± 113.447642 in #0


Epoch #2: 10001it [00:06, 1569.53it/s, env_step=20000, len=500, loss=2.675, n/ep=4, n/st=10, rew=-500.00]                           


Epoch #2: test_reward: -167.090000 ± 88.692964, best_reward: -167.090000 ± 88.692964 in #2


Epoch #3: 10001it [00:24, 413.34it/s, env_step=30000, len=186, loss=3.123, n/ep=0, n/st=10, rew=-185.00]                           


Epoch #3: test_reward: -104.370000 ± 30.535113, best_reward: -104.370000 ± 30.535113 in #3


Epoch #4:  81%|########1 | 8140/10000 [00:42<00:09, 193.61it/s, env_step=38140, len=97, n/ep=1, n/st=10, rew=-96.00]               

Finished training! Use 101.32s


In [24]:
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import BasicLogger

writer = SummaryWriter('log/dqn_Acrobot-v1')
logger = BasicLogger(writer)

/home/jaewon/anaconda3/lib/python3.8/site-packages/tianshou/utils/logger/tensorboard.py:84: UserWarning: Deprecated soon: BasicLogger has renamed to TensorboardLogger in #427.
  warnings.warn(


In [25]:
torch.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(torch.load('dqn.pth'))

<All keys matched successfully>

In [26]:
policy.eval()
policy.set_eps(0.07)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/home/jaewon/anaconda3/lib/python3.8/site-packages/tianshou/data/collector.py:66: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


{'n/ep': 1,
 'n/st': 92,
 'rews': array([-91.]),
 'lens': array([92]),
 'idxs': array([0]),
 'rew': -91.0,
 'len': 92.0,
 'rew_std': 0.0,
 'len_std': 0.0}